Author: Nelson Liu

Email: [nliu@uncharted.software](mailto:nliu@uncharted.software)

# Model Representation Conversion

Interconvert between ODE and Petri-net representation of a given model:
* ODE representation can be in LaTeX or MathML format
* Petri-net representation is in ACSet JSON format

## Petri-to-ODE conversion using SciML

Source: 
* [https://docs.sciml.ai/EasyModelAnalysis/stable/examples/petri/](https://docs.sciml.ai/EasyModelAnalysis/stable/examples/petri/)
* [https://docs.sciml.ai/ModelingToolkit/stable/tutorials/acausal_components/#Connecting-and-Simulating-Our-Electric-Circuit](https://docs.sciml.ai/ModelingToolkit/stable/tutorials/acausal_components/#Connecting-and-Simulating-Our-Electric-Circuit)

In [120]:
# Libraries
using AlgebraicPetri, AlgebraicPetri.SubACSets, AlgebraicPetri.TypedPetri, AlgebraicPetri.BilayerNetworks
using Catlab.Programs, Catlab.Graphics, Catlab.CategoricalAlgebra
import Catlab.CategoricalAlgebra: migrate!
using GraphViz
# using EasyModelAnalysis
using ModelingToolkit

import Pkg
Pkg.add("Latexify")
using Latexify

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [121]:
# Infectious ontology
const infectious_ontology = LabelledPetriNet(
    [:Pop],
    :infect => ((:Pop, :Pop) => (:Pop, :Pop)),
    :disease => (:Pop => :Pop),
    :strata => (:Pop => :Pop)
)

LabelledPetriNet with elements T = 1:3, S = 1:1, I = 1:4, O = 1:4
┌───┬─────────┐
│ T │   tname │
├───┼─────────┤
│ 1 │  infect │
│ 2 │ disease │
│ 3 │  strata │
└───┴─────────┘
┌───┬───────┐
│ S │ sname │
├───┼───────┤
│ 1 │   Pop │
└───┴───────┘
┌───┬────┬────┐
│ I │ it │ is │
├───┼────┼────┤
│ 1 │  1 │  1 │
│ 2 │  1 │  1 │
│ 3 │  2 │  1 │
│ 4 │  3 │  1 │
└───┴────┴────┘
┌───┬────┬────┐
│ O │ ot │ os │
├───┼────┼────┤
│ 1 │  1 │  1 │
│ 2 │  1 │  1 │
│ 3 │  2 │  1 │
│ 4 │  3 │  1 │
└───┴────┴────┘

In [128]:
# Load labelled Petri net of age-contact model
filename = "../multi-stratify-experiment/model_petri_age.json"
# filename = "./model_petri_sirdh.json"
# filename = "./model_petri_sirdh_skema.json"
model = read_json_acset(LabelledPetriNet, filename)

LabelledPetriNet with elements T = 1:4, S = 1:2, I = 1:8, O = 1:8
┌───┬───────┐
│ T │ tname │
├───┼───────┤
│ 1 │ A1_A1 │
│ 2 │ A1_A2 │
│ 3 │ A2_A1 │
│ 4 │ A2_A2 │
└───┴───────┘
┌───┬───────┐
│ S │ sname │
├───┼───────┤
│ 1 │    A1 │
│ 2 │    A2 │
└───┴───────┘
┌───┬────┬────┐
│ I │ it │ is │
├───┼────┼────┤
│ 1 │  1 │  1 │
│ 2 │  1 │  1 │
│ 3 │  2 │  1 │
│ 4 │  2 │  2 │
│ 5 │  3 │  2 │
│ 6 │  3 │  1 │
│ 7 │  4 │  2 │
│ 8 │  4 │  2 │
└───┴────┴────┘
┌───┬────┬────┐
│ O │ ot │ os │
├───┼────┼────┤
│ 1 │  1 │  1 │
│ 2 │  1 │  1 │
│ 3 │  2 │  1 │
│ 4 │  2 │  2 │
│ 5 │  3 │  2 │
│ 6 │  3 │  1 │
│ 7 │  4 │  2 │
│ 8 │  4 │  2 │
└───┴────┴────┘

In [129]:
# Migrate to a bilayer network
# model_bilayer = LabelledBilayerNetwork()
# migrate!(model_bilayer, model)

In [124]:
# Convert to AlgebraicPetri's `LabelledPetri` to SciML's `ODESystem`
# model_odesys = ODESystem(model_bilayer)
model_odesys = ODESystem(model)

Model PetriNet with 2 equations
States (2):
  A1(t)
  A2(t)
Parameters (4):
  A1_A1
  A1_A2
  A2_A1
  A2_A2

Problematic that SciML ODESystem cancels out all the structure of the above age-contact model's Petri net.
That structure is necessary for subsequent stratification.

Expected ODE:
* `d/dt A1 = - 2 c11 A1 A1 + 2 c11 A1 A1 - c12 A1 A2 + c12 A1 A2 - c21 A2 A1 + c21 A2 A1`
* `d/dt A2 = - 2 c22 A2 A2 + 2 c22 A2 A2 + c12 A1 A2 - c12 A1 A2 + c21 A2 A1 - c21 A2 A1`

`infect`-type two-to-two interactions:
* `A1 + A1 --k1(c11)--> A1 + A1`
* `A1 + A2 --k2(c12)--> A1 + A2`
* `A2 + A1 --k3(c21)--> A2 + A1`
* `A2 + A2 --k4(c22)--> A2 + A2`

In [125]:
# Extract LaTeX code of the ODE system
model_latex = latexify(model_odesys)

L"\begin{align}
\frac{\mathrm{d} \mathrm{A1}\left( t \right)}{\mathrm{d}t} =& 0 \\
\frac{\mathrm{d} \mathrm{A2}\left( t \right)}{\mathrm{d}t} =& 0
\end{align}
"

In [126]:
print(model_latex)

\begin{align}
\frac{\mathrm{d} \mathrm{A1}\left( t \right)}{\mathrm{d}t} =& 0 \\
\frac{\mathrm{d} \mathrm{A2}\left( t \right)}{\mathrm{d}t} =& 0
\end{align}


In [127]:
open("./model_latex.tex", "w") do file
    write(file, model_latex)
end

157